In [37]:
import numpy as np
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import LambdaCallback
from keras.utils import np_utils
from keras.optimizers import RMSprop
import pandas as pd
import os
import random

In [51]:
step_length = 1    # The step length we take to get our samples from our corpus
epochs = 100       # Number of times we train on our full data
batch_size = 32    # Data samples in each training step
latent_dim = 64    # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./horse_gen_model.h5') # Location for the model
load_model = False # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1      # Print result for each epoch
gen_amount = 1000    # How many 

In [39]:
#load names
df = pd.read_csv("preprocessed.csv")
name_list = df['Horses'].values.tolist()
name_list = [str(i) for i in name_list]


In [40]:
concat_names = '\n'.join(name_list).lower()

# Find all unique characters by using set()
chars = sorted(list(set(concat_names)))
num_chars = len(chars)

char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}


# Use longest name length as our sequence window
max_sequence_length = max([len(name) for name in name_list])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(name_list))
print('Longest name: ', max_sequence_length)

Total chars: 36
Corpus length: 1477353
Number of names:  152290
Longest name:  33


In [41]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}]   y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 1477320
First 10 sequences and next chars:
X=[darley arabian godolphin arabian ]   y=[b]
X=[arley arabian godolphin arabian b]   y=[y]
X=[rley arabian godolphin arabian by]   y=[e]
X=[ley arabian godolphin arabian bye]   y=[r]
X=[ey arabian godolphin arabian byer]   y=[l]
X=[y arabian godolphin arabian byerl]   y=[e]
X=[ arabian godolphin arabian byerle]   y=[y]
X=[arabian godolphin arabian byerley]   y=[ ]
X=[rabian godolphin arabian byerley ]   y=[t]
X=[abian godolphin arabian byerley t]   y=[u]


In [42]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=bool)
Y = np.zeros((num_sequences, num_chars), dtype=bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char_to_index[char]] = 1
    Y[i, char_to_index[next_chars[i]]] = 1
    
print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))


X shape: (1477320, 33, 36)
Y shape: (1477320, 36)


In [43]:
model = Sequential()
model.add(LSTM(latent_dim, 
               input_shape=(max_sequence_length, num_chars),  
               recurrent_dropout=dropout_rate))
model.add(Dense(units=num_chars, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 64)                25856     
                                                                 
 dense_5 (Dense)             (None, 36)                2340      
                                                                 
Total params: 28,196
Trainable params: 28,196
Non-trainable params: 0
_________________________________________________________________


In [44]:
if load_model:
    model.load_weights(model_path)
else:
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
    
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)



Start training for 100 epochs
Epoch 1/100
46167/46167 [==============================] - 809s 17ms/step - loss: 2.2522
Epoch 2/100
46167/46167 [==============================] - 816s 18ms/step - loss: 2.2141
Epoch 3/100
46167/46167 [==============================] - 827s 18ms/step - loss: 6.8855
Epoch 4/100
46167/46167 [==============================] - 826s 18ms/step - loss: 5.0751
Epoch 5/100
46167/46167 [==============================] - 823s 18ms/step - loss: 3.6776
Epoch 6/100
46167/46167 [==============================] - 821s 18ms/step - loss: 3.3207
Epoch 7/100
46167/46167 [==============================] - 807s 17ms/step - loss: 3.1463
Epoch 8/100
46167/46167 [==============================] - 804s 17ms/step - loss: 3.0746
Epoch 9/100
46167/46167 [==============================] - 812s 18ms/step - loss: 3.0871
Epoch 10/100
46167/46167 [==============================] - 811s 18ms/step - loss: 3.0889
Epoch 11/100
46167/46167 [==============================] - 809s 18ms/step - lo

In [55]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []

print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char_to_index[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)   
    next_char = index_to_char[next_idx]   
    
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':

        gen_name = [name for name in sequence.split('\n')][1]

        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]

        # Discard all names that are too short
        if len(gen_name) > 2:
            
            # Only allow new and unique names
            if gen_name not in name_list + new_names:
                new_names.append(gen_name.capitalize())

        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

1000 new names are being generated
Generated 0
Generated 0
Generated 0
Generated 0
Generated 0
Generated 0
Generated 0
Generated 100
Generated 200
Generated 300
Generated 400
Generated 500
Generated 600
Generated 700
Generated 800
Generated 900
Generated 1000


In [56]:
print_first_n = min(10, gen_amount)

print('First {} generated names:'.format(print_first_n))
for name in new_names[:print_first_n]:
    print(name)

First 10 generated names:
Dolapa
Dolapa
Dolapa
Rd aer
Rd aer
Nt or hy wfi hteg
Arrol
Arrol
Arrol
Arrol


In [57]:
concat_output = '\n'.join(sorted(new_names))
output_path = os.path.realpath('generated_names.txt')

with open(output_path, 'a') as f:
    f.write(concat_output)